In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import time 

from code.experiment_functions import * 

In [ ]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import warnings
import numpy as np

# Create Dataset
np.random.seed(0)
X_ =  np.random.uniform(-1, 1, (40, 4)) #np.random.rand(200, 4)

y_ = np.logical_xor(X_[:, 0] > 0,
                       X_[:, 1] > 0)
y_ = np.where(y_, 1, -1)

fig = plt.figure(figsize=(10,10))
plt.scatter(X_[y_ == 1, 0],
            X_[y_ == 1, 1],
            c='b', marker='x',
            label='1')
plt.scatter(X_[y_ == -1, 0],
            X_[y_ == -1, 1],
            c='r',
            marker='s',
            label='-1')

plt.xlim([-1, 1])
plt.ylim([-1, 1])
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_ = np.round(X_, 2)
X_[X_ == 0] = 0.01
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.33, random_state=42)

In [ ]:
X_train = [list(x) for x in X_train]
X_test = [list(x) for x in X_test]
y_train = [int(1) if x == 1 else int(0) for x in y_train]
y_test = [int(1) if x == 1 else int(0) for x in y_test]

In [ ]:
"""
with open('results/version6/train_nonlinear.json', 'w+') as fp:
    json.dump([X_train, y_train], fp)
    
with open('results/version6/test_nonlinear.json', 'w+') as fp:
    json.dump([X_test, y_test], fp)
"""

#### load saved data

In [2]:
with open('results/version6/train_nonlinear.json') as json_file:
    X_train = json.load(json_file)[0]

with open('results/version6/train_nonlinear.json') as json_file:
    y_train = json.load(json_file)[1]

In [3]:
with open('results/version6/test_nonlinear.json') as json_file:
    X_test = json.load(json_file)[0]

with open('results/version6/test_nonlinear.json') as json_file:
    y_test = json.load(json_file)[1]

In [ ]:
def float_to_binary(list_, m, n):
    """Convert the float value `x` to a binary string of length `m + n`
    where the first `m` binary digits are the integer part and the last
    'n' binary digits are the fractional part of `x`.
    """
    list_scaled = []
    for number in list_:
        x_scaled = round(number * 2 ** n)
        x_scaled = '{:0{}b}'.format(x_scaled, m + n)
        for i in x_scaled:
            if i == '0':
                list_scaled.append(-1)
            else:
                list_scaled.append(1)
    return list_scaled

In [ ]:
X_test_bin = []
for i in X_test:
    X_test_bin.append(float_to_binary(i, 1, 4))

In [ ]:
X_train_bin = []
for i in X_train:
    X_train_bin.append(float_to_binary(i, 1, 4))

### Tests

In [ ]:
w = list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test_bin[0]),)))

In [ ]:
weights = quantumNeuronFIT(X_test_bin, 
                           y_train,
                           init_weight = w,
                           threshold=0.5,
                           lrParameter=0.1, 
                           n_epochs=20, 
                           trainingBias=True,
                           epoch_results=False,
                           trainingApproaches={'hsgs'})

In [ ]:
results = quantumNeuronPREDICT(X_test, 
                                   y_test,
                                   threshold=0.5,
                                   weightVectorsPhaseEncoding = weights[0], 
                                   weightVectorsHSGS = weights[1], 
                                   repeat=10,
                                   bias=False, 
                                   testingApproaches={'hsgs'})

In [ ]:
print('\HSGS avg_error', results[3])
print('HSGS weights', results[5])
print('HSGS neuron_outputs: ', results[4])
print('targets: ', results[6])

## Binary Quantum Neuron Experiments 

In [ ]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [ ]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_train_bin[0]),)))
threshold_space = [0.1, 0.3, 0.5, 0.7, 0.8]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original']

In [ ]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train_bin, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=20, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'hsgs'})

                    results = quantumNeuronPREDICT(X_test_bin, 
                                                   y_test,
                                                   threshold=threshold,
                                                   weightVectorsPhaseEncoding = weights[0], 
                                                   weightVectorsHSGS = weights[1], 
                                                   repeat=10,
                                                   bias=bias, 
                                                   phaseEstrategyOperator=strategy,
                                                   testingApproaches={'hsgs'})


                    output['model'].append('HSGS')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[3])
                    output['neuron_outputs'].append(results[4])
                    output['trained_weights'].append(results[5])
                    output['initial_weights'].append(initial_weights)
                    print('\n')

In [ ]:
experiment_hsgs = pd.DataFrame(output)
experiment_hsgs.to_csv('results/version6/experiment_nonlinear_hsgs.csv')

In [ ]:
experiment_hsgs

## Continuously Valued Quantum Neuron Experiments

In [4]:
output = {'model':[],
          'phase_strategy':[],
            'bias':[],
            'threshold':[],
            'lr':[],
            'avg_error':[],
            'trained_weights':[],
            'initial_weights':[],
            'neuron_outputs':[]}

In [5]:
# search space
initial_weights =  list(np.random.uniform(low=0.0, high=1.0, size=(len(X_test[0]),)))
threshold_space = [0.3, 0.5, 0.7, 0.8]
lr_space = [0.02, 0.1]
bias_space = [False, True]
strategy_space = ['original', 'angle', 'radius', 'angleradius']

In [6]:
%%time
for w in initial_weights:
    for strategy in strategy_space:
        for threshold in threshold_space:
            for lr in lr_space:
                for bias in bias_space:
                    # execute experiment
                    weights = quantumNeuronFIT(X_train, 
                                               y_train,
                                               init_weight=initial_weights,
                                               threshold=threshold,
                                               lrParameter=lr, 
                                               n_epochs=30, 
                                               trainingBias=bias,
                                               phaseEstrategyOperator=strategy,
                                               trainingApproaches={'phase-encoding'})

                    results = quantumNeuronPREDICT(X_test, 
                                                       y_test,
                                                       threshold=threshold,
                                                       weightVectorsPhaseEncoding = weights[0], 
                                                       weightVectorsHSGS = weights[1], 
                                                       repeat=10,
                                                       bias=bias, 
                                                       phaseEstrategyOperator=strategy,
                                                       testingApproaches={'phase-encoding'})

                    # get experiment results
                    output['model'].append('phase-encoding')
                    output['lr'].append(lr)
                    output['bias'].append(bias)
                    output['threshold'].append(threshold)
                    output['phase_strategy'].append(strategy)
                    output['avg_error'].append(results[0])
                    output['neuron_outputs'].append(results[1])
                    output['trained_weights'].append(results[2])
                    output['initial_weights'].append(w)
                    print('\n')

lrParameter:  0.02
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  4
AVG TEST ERROR PHASE   0.28569999999999995


lrParameter:  0.02
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original


/opt/conda/lib/python3.7/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)



best error phase-encoding training:  4
AVG TEST ERROR PHASE   0.4286


lrParameter:  0.1
threshold:  0.3
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  5
AVG TEST ERROR PHASE   0.28569999999999995


lrParameter:  0.1
threshold:  0.3
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  4
AVG TEST ERROR PHASE   0.4286


lrParameter:  0.02
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  4
AVG TEST ERROR PHASE   0.1429


lrParameter:  0.02
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  original

best error phase-encoding training:  4
AVG TEST ERROR PHASE   0.4286


lrParameter:  0.1
threshold:  0.5
trainingBias:  False
phaseEstrategyOperator:  original

best error phase-encoding training:  6
AVG TEST ERROR PHASE   0.28569999999999995


lrParameter:  0.1
threshold:  0.5
trainingBias:  True
phaseEstrategyOperator:  original

best err

In [7]:
experiment_phase = pd.DataFrame(output)
experiment_phase.to_csv('results/version6/experiment_nonlinear_phase_tanh.csv')